## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key
# ^^^ Had to code it like this because "from config import g_key" alone would result in an error.

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kontokalion,GR,39.6453,19.8494,66.06,82,20,0.00,few clouds
1,1,Euclides Da Cunha,BR,-10.5075,-39.0158,68.22,97,100,3.51,overcast clouds
2,2,Yellowknife,CA,62.4560,-114.3525,52.32,37,75,11.50,broken clouds
3,3,Mataura,NZ,-46.1927,168.8643,54.48,91,100,0.72,overcast clouds
4,4,Albany,US,42.6001,-73.9662,57.45,81,60,1.01,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
listed_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
listed_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Meiktila,MM,20.8667,95.8667,91.98,49,54,14.03,broken clouds
10,10,Suzhou,CN,31.3114,120.6181,79.68,29,0,8.01,clear sky
13,13,Saryozek,KZ,44.3583,77.9753,70.61,34,40,6.31,scattered clouds
14,14,Harper,LR,4.3750,-7.7169,78.82,82,100,4.14,overcast clouds
15,15,Urumqi,CN,31.3114,120.6181,79.68,29,0,8.01,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
print(len(listed_cities))
listed_cities.count()

216


City_ID                216
City                   216
Country                213
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = listed_cities.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

213


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Meiktila,MM,20.8667,95.8667,91.98,49,54,14.03,broken clouds
10,10,Suzhou,CN,31.3114,120.6181,79.68,29,0,8.01,clear sky
13,13,Saryozek,KZ,44.3583,77.9753,70.61,34,40,6.31,scattered clouds
14,14,Harper,LR,4.3750,-7.7169,78.82,82,100,4.14,overcast clouds
15,15,Urumqi,CN,31.3114,120.6181,79.68,29,0,8.01,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Meiktila,MM,91.98,broken clouds,20.8667,95.8667,
10,Suzhou,CN,79.68,clear sky,31.3114,120.6181,
13,Saryozek,KZ,70.61,scattered clouds,44.3583,77.9753,
14,Harper,LR,78.82,overcast clouds,4.3750,-7.7169,
15,Urumqi,CN,79.68,clear sky,31.3114,120.6181,
19,Loikaw,MM,91.98,broken clouds,20.8667,95.8667,
21,Kharan,PK,86.14,clear sky,28.5833,65.4167,
23,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
26,Jodhpur,IN,90.70,clear sky,26.2867,73.0300,
30,Leh,IN,90.70,clear sky,26.2867,73.0300,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Meiktila,MM,91.98,broken clouds,20.8667,95.8667,The Floral Breeze Wunzin Hotel
10,Suzhou,CN,79.68,clear sky,31.3114,120.6181,Holiday Inn Jasmine Suzhou
13,Saryozek,KZ,70.61,scattered clouds,44.3583,77.9753,Gostinitsa
14,Harper,LR,78.82,overcast clouds,4.3750,-7.7169,Screensaver
15,Urumqi,CN,79.68,clear sky,31.3114,120.6181,Holiday Inn Jasmine Suzhou
19,Loikaw,MM,91.98,broken clouds,20.8667,95.8667,The Floral Breeze Wunzin Hotel
21,Kharan,PK,86.14,clear sky,28.5833,65.4167,Dil Aram Bangla
23,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
26,Jodhpur,IN,90.70,clear sky,26.2867,73.0300,"Umaid Bhawan Palace, Jodhpur"
30,Leh,IN,90.70,clear sky,26.2867,73.0300,"Umaid Bhawan Palace, Jodhpur"


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))